# Run finetuning with my data

In [ ]:
import gpt_2_simple as gpt2
import os
import requests

sess = gpt2.start_tf_sess()

In [ ]:
#model_name = "124M"
model_name = "355M"
#model_name = "774M"
if not os.path.isdir(os.path.join("models", model_name)):
    print(f"Downloading {model_name} model...")
    gpt2.download_gpt2(model_name=model_name)   # model is saved into current directory under /models/124M/

In [ ]:
#Train model for me
name = 'george'
file_name = "discord_logs_just_my_tokens.txt"

gpt2.finetune(sess,
              file_name,
              model_name=model_name,
              steps=1000,
              run_name=name,
              save_every=250,
              overwrite=True)   # steps is max number of training steps

In [ ]:
#Train model for alen
name = 'alen'
file_name = "discord_logs_just_alen_tokens.txt"

gpt2.finetune(sess,
              file_name,
              model_name=model_name,
              steps=1000,
              run_name=name,
              save_every=250,
              overwrite=True)   # steps is max number of training steps

In [ ]:
gpt2.load_gpt2(sess)

In [ ]:
k=20
p=.9
t=.5
max_history=4

history = []
start_text = "What do you think of President Trump?"

bots = ['george', 'alen']
start_tokens = ['<|BrainSquid#1538:|>', '<|alen#8819:|>']

print('Topic = ' + start_text)
history.append(start_text)
counter = 0
error_count = 0
while True:
    
    if counter == 0:
        out_ids = gpt2.generate(sess, run_name=bots[counter], prefix=' '.join(history) + " " + start_tokens[counter], truncate="<|endoftext|>", temperature=t, top_k=k, top_p=p, include_prefix=False, return_as_list=True)
    else:
        out_ids = gpt2.generate(sess, run_name=bots[counter], prefix=' '.join(history) + " " + start_tokens[counter], truncate="<|endoftext|>", temperature=t, top_k=k, top_p=p, include_prefix=False, return_as_list=True)
    #Append output to history
    output = ' '.join(out_ids)
    
    for start_token in start_tokens:
        output = output.split(start_token)[0]
    
    if sum([x in output for x in history]) == len(history):
        #retry
        error_count += 1
        print('ERROR ' + str(error_count), end='\r')
        continue
        
    history.append(output)
    history = history[-(2*max_history+1):]
    print("BOT " + str(bots[counter]) + ' : ' + output)
    counter = (counter + 1) % 2

In [ ]:
k=0
p=.9
t=.5
max_history=4

start_token = '<|BrainSquid#1538:|>'
start_token2 = '<|alen#8819:|>'
history = []
start_text = "What do you think of President Trump?"
print('Topic = ' + start_text)
history.append(start_text)
counter = 0
while True:
    
    if counter:
        out_ids = gpt2.generate(sess, run_name='george', prefix=' '.join(history) + " " + start_token, truncate="<|endoftext|>", temperature=t, top_k=k, top_p=p, include_prefix=False, return_as_list=True)
    else:
        out_ids = gpt2.generate(sess, run_name='alen', prefix=' '.join(history) + " " + start_token2, truncate="<|endoftext|>", temperature=t, top_k=k, top_p=p, include_prefix=False, return_as_list=True)
    #Append output to history
    output = ' '.join(out_ids)
    output = output.split(start_token)[0]
    output = output.split(start_token2)[0]
    
    if sum([x in output for x in history]) == len(history):
        #retry
        print('ERROR')
        continue
        
    history.append(output)
    history = history[-(2*max_history+1):]
    print("Bot " + str(counter) + ' : ' + output)
    counter = (counter + 1) % 2